IMPORT BIBLIOTEK

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)

2.18.0


ŁADOWANIE DANYCH

In [ ]:
mnist_dataset, mnist_info = tfds.load(name ='mnist', with_info=True, as_supervised=True);

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.CW6267_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.CW6267_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [ ]:
num_valid_samples = 0.1 * mnist_info.splits['train'].num_examples
num_valid_samples = tf.cast(num_valid_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [ ]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFSIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFSIZE)
valid_data = shuffled_train_and_validation_data.take(num_valid_samples)
train_data = shuffled_train_and_validation_data.skip(num_valid_samples)
test_data = test_data.batch(num_test_samples)


In [ ]:
test_data = test_data.batch(num_test_samples)
valid_data = valid_data.batch(num_valid_samples)
train_data = train_data.batch(100)
valid_inputs, valid_targets = next(iter(valid_data))
print (valid_inputs.shape)
print (valid_targets.shape)


(6000, 28, 28, 1)
(6000,)


MODEL

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), #warstwa wejść
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #warstwa ukryta
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #warstwa ukryta
    tf.keras.layers.Dense(output_size, activation='softmax') #warstwa wyjść
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
NUM_EPOCHS = 10
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(valid_inputs, valid_targets), verbose=2)


Epoch 1/10
540/540 - 10s - 18ms/step - accuracy: 0.9032 - loss: 0.3380 - val_accuracy: 0.9488 - val_loss: 0.1762
Epoch 2/10
540/540 - 4s - 8ms/step - accuracy: 0.9586 - loss: 0.1413 - val_accuracy: 0.9643 - val_loss: 0.1186
Epoch 3/10
540/540 - 5s - 9ms/step - accuracy: 0.9705 - loss: 0.0999 - val_accuracy: 0.9697 - val_loss: 0.0994
Epoch 4/10
540/540 - 5s - 10ms/step - accuracy: 0.9763 - loss: 0.0787 - val_accuracy: 0.9742 - val_loss: 0.0872
Epoch 5/10
540/540 - 5s - 8ms/step - accuracy: 0.9812 - loss: 0.0618 - val_accuracy: 0.9783 - val_loss: 0.0685
Epoch 6/10
540/540 - 5s - 9ms/step - accuracy: 0.9847 - loss: 0.0512 - val_accuracy: 0.9798 - val_loss: 0.0619
Epoch 7/10
540/540 - 4s - 8ms/step - accuracy: 0.9872 - loss: 0.0415 - val_accuracy: 0.9818 - val_loss: 0.0566
Epoch 8/10
540/540 - 5s - 9ms/step - accuracy: 0.9883 - loss: 0.0381 - val_accuracy: 0.9878 - val_loss: 0.0441
Epoch 9/10
540/540 - 5s - 10ms/step - accuracy: 0.9903 - loss: 0.0312 - val_accuracy: 0.9893 - val_loss: 0.03

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss:{0:.2f}. Test accuracy:{1:.2f}%'.format(test_loss, test_accuracy*100))


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None